In [1]:
import watertap as wt
import pandas as pd
from pyomo.environ import value, Block
from idaes.core import FlowsheetBlock
%matplotlib inline
from pylab import *
import numpy as np
import matplotlib.pyplot as plt
from case_study_trains import *
import case_study_trains

In [ ]:
def it_gets_results(appended, print_it=False, write_it=False):    
    train = wt.case_study_trains.train

    py_var = [
        "fixed_cap_inv",
        "total_cap_investment",
        "cat_and_chem_cost",
        "electricity_cost",
        "total_fixed_op_cost",
    "flow_in", "flow_out"]

    excel_var = ['Fixed Capital Investment (FCI)',
                'Total Capital Investment (TCI)',
                'Catalysts and Chemicals',
                'Electricity',
                'Total Fixed Operating Costs']

    case_study = train['case_study']
    if print_it:
        print(f'\n######### Case study = {case_study} #########\n\n\n')

    cap_inv_tot = []

    big_dict = {}

    for b_unit in m.fs.component_objects(Block, descend_into=True):
        unit = str(b_unit)[3:]
        if hasattr(b_unit, 'costing'):
            if print_it:
                print(f'____________{unit}____________')
            py_vals =  []
            excel_vals = []
            big_dict[unit] = {'python': {}, 'excel': {}}
            df = pd.read_csv("data/case_study_results.csv")
            df = df[df.case_study == train['case_study']]
            df = df[df.scenario == train['scenario']]
            df = df[df.unit_process == unit]
            flow_in_excel = round(df[df.variable == "flow_vol_in"].value.sum(), 5)
            flow_out_excel = round(df[df.variable == 'flow_vol_out'].value.sum(), 5)
            flow_in_python = round(value(b_unit.inlet.flow_vol[0.0] * 3600), 5)
            flow_out_python = round(value(b_unit.outlet.flow_vol[0.0] * 3600), 5)
            waste_python = round(value(b_unit.waste.flow_vol[0.0] * 3600), 5)
            if print_it:
                print(f'\n#### flow_in\nPython --> {flow_in_python}')
                print(f'Excel --> {flow_in_excel}\n\n')
                print(f'#### flow out\nPython --> {flow_out_python}')
                print(f'Excel --> {flow_out_excel}\n\n')
                print(f'#### waste\nPython--> {waste_python}\n')
            for py, ex in dict(zip(py_var, excel_var)).items():
                if print_it:
                    print(f'\n#### {py}')
                try:
                    num = value(getattr(b_unit.costing, py))
                    py_vals.append(num)
                    if print_it:
                        print(f'Python --> {round(num, 5)}')
                except ZeroDivisionError:
                    if print_it:
                        print(f'Python --> {0} - ERROR')
                    py_vals.append(0)

                df = pd.read_csv("data/case_study_results.csv")
                df = df[df.case_study == train['case_study']]
                df = df[df.scenario == train['scenario']]
                df = df[df.unit_process == unit]
                num = df[df.Variable == ex].value.sum()
                if print_it:
                    print(f'Excel --> {round(num, 5)}\n\n')
                excel_vals.append(num)
            py_vals.append(flow_in_python)
            py_vals.append(flow_out_python)
            py_dict = dict(zip(py_var, py_vals))
            excel_vals.append(flow_in_excel)
            excel_vals.append(flow_out_excel)
            excel_dict = dict(zip(py_var, excel_vals))
            big_dict[unit]['python'] = py_dict
            big_dict[unit]['excel'] = excel_dict

    else:
        pass 

    df = pd.DataFrame.from_dict({(i,j): big_dict[i][j] 
                               for i in big_dict.keys() 
                               for j in big_dict[i].keys()},
                               orient='index')

    df['case_study'] = case_study
    df['LCOW'] = value(m.fs.costing.LCOW)
    if write_it:
        df.to_csv(f'results/test_compare_{case_study}_{appended}.csv')

In [3]:
def do_it_all(append1, append2, print_it=False, write_it=True, solver_results=False, model_results=False):
    case_studies = ['Ashkelon', 'Big_Spring', 'Carlsbad', 'EMWD', 'HRSD', 'Irwin', 'KBHDP', 'OCWD', 'Santa_Barbara', 'Solaire', 'Tampa_Bay']
    
    water_sources = ['Seawater', 'Big_Spring_Feed', 'Seawater', 'EMWD_CA_Brackish', 'HRSD_Municipal', 'Brackish', 'KBHDP_Brackish_Ave', 'OCWD_Feed', 'Seawater', 'Solaire_Graywater', 'Seawater']
    
    wt_dict = dict(zip(case_studies, water_sources))
    
    
    for k, v in wt_dict.items():
        m = wt.watertap_setup(dynamic = False)
        try:
        
            wt.case_study_trains.train = {"case_study": k,
                                     "reference": "NAWI",
                                     "scenario": "Baseline"}

            wt.case_study_trains.source_water = {"case_study": k, 
                                 "reference": "NAWI",
                                 "scenario": "Baseline",
                                 "water_type": v}
            
            m = wt.case_study_trains.get_case_study(m=m) 
            
        except:
            print(f'\n\n{k} case study with {v} source water threw an error when trying to build the model! How annoying!\n\n____________________________________________________________________')
        
        try:
            wt.run_water_tap(m=m, solver_results=solver_results, print_model_results=model_results)
        
            it_gets_results(append1, print_it=print_it, write_it=write_it)
        
            wt.run_water_tap(m=m, solver_results=solver_results, print_model_results=model_results)
        
            it_gets_results(append2, print_it=print_it, write_it=write_it)
        
        except:
            print(f'\n\n{k} case study with {v} source water threw an error when trying to solve! How annoying!\n\n____________________________________________________________________')


In [ ]:
do_it_all('a','b', write_it=False)

Ashkelon
sw_onshore_intake
sulfuric_acid_addition
ferric_chloride_addition
chlorination
static_mixer
tri_media_filtration
reverse_osmosis
lime_softening
chlorination_b
caustic_soda_addition
treated_storage
backwash_solids_handling
municipal_drinking
surface_discharge
adding source
degrees_of_freedom: 10
Ipopt 3.12.12: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.12, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     2551
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros 